In [ ]:
!unzip /content/RAVDESS_dataset -d /content/RAVDESS_dataset/

In [2]:
!pip install audiomentations

In [14]:
# OS BASED PATH
# base = Path(__file__).resolve().parent
# Notebook Path
import librosa
import soundfile
import os
import glob
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler
import audiomentations


# All files and directories ending with .txt and that don't begin with a dot:
ROOT = os.path.abspath('')
dataset_path = os.path.join(ROOT, 'RAVDESS_dataset')

dataset_files = glob.glob(f"{dataset_path}/Actor_*/*.wav")

# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
focused_emotions=['neutral', 'calm', 'happy', 'sad', 'disgust', 'angry']

sample_rate = 16000

In [15]:
# Extract features (mfcc, chroma, mel) from a sound file
# Additional features for feature extraction
def extract_feature(file_name, mfcc, chroma, mel, contrast, tonnetz, zcr):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
            result = np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T, axis=0)
            result = np.hstack((result, tonnetz))
        if zcr:
            zcr = np.mean(librosa.feature.zero_crossing_rate(y=X).T, axis=0)
            result = np.hstack((result, zcr))
    return result

# Augmentation function
def augment_data(x, y, sample_rate, aug_factor=2):
    augmented_data = []
    augmented_labels = []
    aug = audiomentations.Compose([
        audiomentations.AddGaussianNoise(p=0.5),
        audiomentations.TimeStretch(p=0.5),
        audiomentations.PitchShift(p=0.5),
    ])
    for i in range(len(x)):
        for _ in range(aug_factor):
            augmented_data.append(aug(x[i], sample_rate=sample_rate))
            augmented_labels.append(y[i])
    return np.array(augmented_data), np.array(augmented_labels)

# Extract features with additional options
def load_data(test_size=0.2):
    x, y = [], []
    for file in dataset_files:
        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]
        if emotion not in focused_emotions:
            continue
        feature = extract_feature(file, mfcc=True, chroma=True, mel=True, contrast=True, tonnetz=True, zcr=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [16]:
# Load and preprocess the data
x_train, x_test, y_train, y_test = load_data(test_size=0.25)

# Augment the training data
x_train_augmented, y_train_augmented = augment_data(x_train, y_train, sample_rate)

# Combine original and augmented data
x_train_combined = np.concatenate((x_train, x_train_augmented))
y_train_combined = np.concatenate((y_train, y_train_augmented))

#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

/usr/local/lib/python3.10/dist-packages/audiomentations/core/transforms_interface.py:62: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=194
  warnings.warn(


(792, 264)
Features extracted: 194


In [17]:
# Standardize features
scaler = StandardScaler()
x_train_combined_scaled = scaler.fit_transform(x_train_combined)
x_test_scaled = scaler.transform(x_test)

# Initialize the Multi-Layer Perceptron Classifier with adjusted parameters
model = MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(500,), learning_rate='adaptive', max_iter=500)

# Train the model
model.fit(x_train_combined_scaled, y_train_combined)

# Predict for the test set
y_pred = model.predict(x_test_scaled)

# Calculate the accuracy of our model
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)

# Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 69.70%


In [18]:
y_pred

array(['sad', 'neutral', 'neutral', 'sad', 'sad', 'angry', 'calm', 'calm',
       'neutral', 'neutral', 'angry', 'neutral', 'happy', 'neutral',
       'neutral', 'neutral', 'calm', 'sad', 'happy', 'disgust', 'happy',
       'sad', 'happy', 'happy', 'happy', 'calm', 'angry', 'neutral',
       'neutral', 'disgust', 'calm', 'sad', 'sad', 'disgust', 'happy',
       'neutral', 'happy', 'neutral', 'happy', 'happy', 'calm', 'calm',
       'sad', 'disgust', 'happy', 'neutral', 'angry', 'angry', 'disgust',
       'calm', 'calm', 'calm', 'neutral', 'calm', 'calm', 'sad', 'angry',
       'happy', 'disgust', 'happy', 'disgust', 'sad', 'disgust', 'happy',
       'sad', 'disgust', 'calm', 'calm', 'happy', 'neutral', 'calm',
       'sad', 'angry', 'happy', 'calm', 'angry', 'happy', 'disgust',
       'calm', 'happy', 'happy', 'sad', 'disgust', 'disgust', 'angry',
       'happy', 'happy', 'angry', 'happy', 'angry', 'sad', 'calm',
       'happy', 'disgust', 'happy', 'angry', 'calm', 'disgust', 'calm',
 

In [19]:
f1_score(y_test, y_pred,average=None)

array([0.76190476, 0.78787879, 0.68      , 0.67272727, 0.625     ,
       0.62068966])

In [20]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,sad,sad
1,neutral,neutral
2,neutral,neutral
3,sad,sad
4,happy,sad
5,angry,angry
6,calm,calm
7,calm,calm
8,neutral,neutral
9,sad,neutral


In [21]:
# import pickle
# # Writing different model files to file
# with open( 'modelForPrediction1.sav', 'wb') as f:
#     pickle.dump(model,f)
import joblib

clf_file = 'emotion_classifier_v2.pkl'

joblib.dump(model, clf_file)

['emotion_classifier_v2.pkl']

In [23]:
filename = 'emotion_classifier_v2.pkl'
loaded_model = joblib.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature(f"{dataset_path}/Actor_01/03-01-01-01-01-01-01.wav", mfcc=True, chroma=True, mel=True, contrast=True, tonnetz=True, zcr=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['sad'], dtype='<U7')

In [ ]:
feature